In [51]:
import pandas as pd
import numpy as np
import os
import re
import ast

In [52]:
def merge_csv_files(folder_path, output_file):
    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

    merged_data = pd.DataFrame()

    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        merged_data = pd.concat([merged_data, df], ignore_index=True)

    merged_data.to_csv(output_file, index=False)
    print(f"Merged data saved to {output_file}.")

In [53]:
if __name__ == "__main__":
    data_folder = "MagnaOpus_GitHub/transform/housing3"
    output_csv_file = "MagnaOpus_GitHub/transform/merged/housing3_merged.csv"
    merge_csv_files(data_folder, output_csv_file)

Merged data saved to /Users/aryrubi/Desktop/Python_projects/MagnaOpus/cleaning/merged/housing3_merged.csv.


In [54]:
df = pd.read_csv(output_csv_file)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875 entries, 0 to 874
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              875 non-null    object
 1   title           875 non-null    object
 2   location        875 non-null    object
 3   price           875 non-null    object
 4   details         875 non-null    object
 5   specifications  875 non-null    object
dtypes: object(6)
memory usage: 41.1+ KB


In [55]:
def convert_to_list(string_data):
    return ast.literal_eval(string_data)

# Apply the conversion function to the entire column
df['details'] = df['details'].apply(convert_to_list)
df['specifications'] = df['specifications'].apply(convert_to_list)

In [56]:
def find_estacionamiento(specs):
    for spec in specs:
        if 'Estacionamiento' in spec:
            # Extracting the numeric value using regular expressions
            value = re.search(r'\d+', spec)
            if value:
                return int(value.group(0))
    return 0  # Return 0 if Estacionamiento is not found

In [57]:
max_length = df['details'].apply(len).max()

for i in range(max_length):
    df[f'element_{i + 1}'] = df['details'].apply(lambda x: x[i] if len(x) > i else None)

df['estacionamiento'] = df['specifications'].apply(find_estacionamiento)

df.drop(columns=['details', 'specifications'], inplace=True)


In [58]:
df

,ID,title,location,price,element_1,element_2,element_3,estacionamiento
0,ASCGI6U,VENTA DE CASA EN COLONIA MIRAMONTES,Colonia Miramontes,\n $ 323...,Habitaciones3,Baños3.5,Área325 m2,1
1,GH5NCL5,VENTA DE CASA EN RESIDENCIAL COLONIA TEPEYAC,Colonia Tepeyac,\n $ 976...,Habitaciones4,Baños5.5,Área620 m2,2
2,GZ3RDZO,VENTA DE CASA EN CASTAÑOS SUR,Castaños Sur,\n $ 243...,Habitaciones4,Baños3.5,Área665 m2,2
3,SX9J5M9,VENTA DE CASA EN QUINTA BELLA,Colonia San Ignacio,\n $ 440...,Habitaciones4,Baños3.5,Área400 m2,0
4,V8GBXZK,VENTA DE CASAS EN COLONIA 15 DE SEPTIEMBRE,Colonia 15 de Septiembre,\n $ 263...,Habitaciones5,Baños3,Área336 m2,2
...,...,...,...,...,...,...,...,...
870,JMJ52O2,SE VENDE CASA CERCA DEL PARQUE ACUÁTICO AQUA S...,Carretera Vieja del Norte,\n $ 153...,Habitaciones3,Baños4,Área232 m2,0
871,QF2OG3A,SE VENDE CASA EN RESIDENCIAL CERRO AZUL,Tegucigalpa,\n $ 180...,Área167 m2,None,None,0
872,9ER8DEQ,VENTA DE CASA EN EL HATILLO,Tegucigalpa,\n $ 290...,Habitaciones5,Baños4,Área301.74 m2,5
873,9ER8DEQ,VENTA DE CASA EN EL HATILLO,Tegucigalpa,\n $ 290...,Habitaciones5,Baños4,Área301.74 m2,5


In [59]:
new_column_names = {'element_1': 'Habitaciones',
                    'element_2': 'Baños',
                    'element_3': 'Área'}
df = df.rename(columns=new_column_names)

In [60]:
df['Habitaciones'] = df['Habitaciones'].astype(str)
df['Baños'] = df['Baños'].astype(str)
df['Área'] = df['Área'].astype(str)

In [61]:
string_columns = df.select_dtypes(include=[object]).columns
df[string_columns] = df[string_columns].applymap(str.strip)

In [62]:
def organize_data(df):
    data_list = []
    max_length = 0

    for index, row in df.iterrows():
        new_data = {}
        for column in df.columns:
            new_data[column] = []

        for column, value in row.items():
            if isinstance(value, str):
                for new_column in new_data.keys():
                    if value.startswith(new_column):
                        new_data[new_column].append(value.replace(new_column, '').strip())
                        break
                else:
                    new_data[column].append(value)
            else:
                new_data[column].append(value if isinstance(value, list) else [value] * max_length)

        data_list.append(new_data)

    return pd.DataFrame(data_list)


In [63]:
def extract_single_value_or_none(df):
    """
    Extract the single value from each list or assign None if the list is empty in each column of the DataFrame.

    Parameters:
        df (pd.DataFrame): The DataFrame to extract single values from.

    Returns:
        pd.DataFrame: A new DataFrame with single values extracted or None if the list is empty.
    """
    extracted_data = {}

    for column in df.columns:
        extracted_data[column] = df[column].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)

    return pd.DataFrame(extracted_data)

In [64]:
organized_df = organize_data(df)
extracted_df = extract_single_value_or_none(organized_df)

In [65]:
extracted_df

,ID,title,location,price,Habitaciones,Baños,Área,estacionamiento
0,ASCGI6U,VENTA DE CASA EN COLONIA MIRAMONTES,Colonia Miramontes,"$ 323,631",3,3.5,325 m2,[]
1,GH5NCL5,VENTA DE CASA EN RESIDENCIAL COLONIA TEPEYAC,Colonia Tepeyac,"$ 976,000",4,5.5,620 m2,[]
2,GZ3RDZO,VENTA DE CASA EN CASTAÑOS SUR,Castaños Sur,"$ 243,000",4,3.5,665 m2,[]
3,SX9J5M9,VENTA DE CASA EN QUINTA BELLA,Colonia San Ignacio,"$ 440,000",4,3.5,400 m2,[]
4,V8GBXZK,VENTA DE CASAS EN COLONIA 15 DE SEPTIEMBRE,Colonia 15 de Septiembre,"$ 263,188",5,3,336 m2,[]
...,...,...,...,...,...,...,...,...
870,JMJ52O2,SE VENDE CASA CERCA DEL PARQUE ACUÁTICO AQUA S...,Carretera Vieja del Norte,"$ 153,507",3,4,232 m2,[]
871,QF2OG3A,SE VENDE CASA EN RESIDENCIAL CERRO AZUL,Tegucigalpa,"$ 180,294",None,None,167 m2,[]
872,9ER8DEQ,VENTA DE CASA EN EL HATILLO,Tegucigalpa,"$ 290,000",5,4,301.74 m2,[]
873,9ER8DEQ,VENTA DE CASA EN EL HATILLO,Tegucigalpa,"$ 290,000",5,4,301.74 m2,[]


In [66]:
extracted_df['price'] = extracted_df['price'].str.replace('[\$,]', '', regex=True).astype(float)
extracted_df['Habitaciones'] = extracted_df['Habitaciones'].astype(str).apply(lambda x: re.search(r'\d+', x).group() if re.search(r'\d+', x) else None)
extracted_df['Baños'] = extracted_df['Baños'].astype(str).apply(lambda x: re.search(r'\d+', x).group() if re.search(r'\d+', x) else None)
extracted_df['Área'] = extracted_df['Área'].astype(str).apply(lambda x: re.search(r'\d+', x).group() if re.search(r'\d+', x) else None)

In [67]:
extracted_df.isna().sum()

ID                   0
title                0
location             0
price                0
Habitaciones        60
Baños              161
Área               227
estacionamiento      0
dtype: int64

In [68]:
clean_df = extracted_df.dropna()

In [69]:
clean_df['Área'] = clean_df['Área'].str.replace(r'[^0-9.-]', '', regex=True).astype(float)
clean_df['Habitaciones'] = clean_df['Habitaciones'].str.replace(r'[^0-9.-]', '', regex=True).astype(float)
clean_df['Baños'] = clean_df['Baños'].str.replace(r'[^0-9.-]', '', regex=True).astype(float)

/var/folders/j2/rnd6rd7x5fdfxh0vj53x3fqh0000gn/T/ipykernel_97339/2940481360.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['Área'] = clean_df['Área'].str.replace(r'[^0-9.-]', '', regex=True).astype(float)
/var/folders/j2/rnd6rd7x5fdfxh0vj53x3fqh0000gn/T/ipykernel_97339/2940481360.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['Habitaciones'] = clean_df['Habitaciones'].str.replace(r'[^0-9.-]', '', regex=True).astype(float)
/var/folders/j2/rnd6rd7x5fdfxh0vj53x3fqh0000gn/T/ipykernel

In [70]:
final_df = clean_df.drop(columns=['title'])

In [71]:
final_df.rename(columns={'price': 'usd_price','Habitaciones': 'bedrooms', 'Baños': 'bathrooms', 'Área': 'surface_m²', 'estacionamiento': 'parking'}, inplace=True)

final_df['usd_price'] = final_df['usd_price'].astype(float)
final_df['bedrooms'] = final_df['bedrooms'].astype(int)
final_df['bathrooms'] = final_df['bathrooms'].astype(float)
final_df['surface_m²'] = final_df['surface_m²'].astype(float)

In [72]:
duplicates = final_df[final_df['ID'].duplicated()]

# Dropping rows with duplicated IDs
df_unique = final_df[~final_df['ID'].duplicated()]
df_unique = df_unique.drop(columns=['ID'])

In [73]:
df_unique['parking'] = df['estacionamiento']

In [74]:
df_unique.to_csv("MagnaOpus_GitHub/cleaning/data/web3_transformed_data.csv", index = False)